In [11]:
import heapq

class PuzzleState:
    def __init__(self, board, parent=None, move="", depth=0, cost=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

# ✅ Fixed get_manhattan_distance function
def get_manhattan_distance(board, goal):
    """Calculates Manhattan Distance heuristic."""
    distance = 0
    flat_goal = sum(goal, [])  # Flatten the goal state (2D → 1D)

    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:  # Ignore empty tile
                continue
            x, y = divmod(flat_goal.index(board[i][j]), 3)  # Correct position lookup
            distance += abs(i - x) + abs(j - y)

    return distance

def get_neighbors(state, goal_state):
    """Generates possible next moves."""
    neighbors = []
    board = state.board
    x, y = [(r, c) for r in range(3) for c in range(3) if board[r][c] == 0][0]

    moves = {"Up": (-1, 0), "Down": (1, 0), "Left": (0, -1), "Right": (0, 1)}

    for move, (dx, dy) in moves.items():
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < 3 and 0 <= new_y < 3:
            new_board = [row[:] for row in board]
            new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
            new_state = PuzzleState(
                new_board, state, move, state.depth + 1,
                state.depth + 1 + get_manhattan_distance(new_board, goal_state)
            )
            neighbors.append(new_state)

    return neighbors

def solve_8_puzzle(initial_state, goal_state):
    """A* Algorithm to solve the 8-Puzzle"""
    start = PuzzleState(initial_state, cost=get_manhattan_distance(initial_state, goal_state))
    open_list = []
    heapq.heappush(open_list, start)
    closed_set = set()

    while open_list:
        current_state = heapq.heappop(open_list)

        if current_state.board == goal_state:
            path = []
            while current_state.parent:
                path.append(current_state.move)
                current_state = current_state.parent
            return path[::-1]  # Return moves from start to goal

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in get_neighbors(current_state, goal_state):
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue
            heapq.heappush(open_list, neighbor)

    return None  # No solution found

def print_board(board):
    """Prints the board in a readable format."""
    for row in board:
        print(" ".join(map(str, row)))
    print("\n")

# Example Run
initial_state = [
    [1, 2, 3],
    [5, 6, 0],
    [7, 8, 4]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

print("Initial State:")
print_board(initial_state)

solution = solve_8_puzzle(initial_state, goal_state)

if solution:
    print("Solution Found!\nMoves:", " -> ".join(solution))
else:
    print("No solution exists!")


Initial State:
1 2 3
5 6 0
7 8 4


Solution Found!
Moves: Left -> Left -> Down -> Right -> Right -> Up -> Left -> Down -> Left -> Up -> Right -> Right -> Down
